In [1]:
import numpy as np
import numpy.linalg as la
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm

import osmnx as ox
import momepy
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import alphashape
from pyproj import Proj, Geod
import ast
from shapely.ops import cascaded_union, polygonize, unary_union

data_path = '../../data/'  
polygon_road_network = gpd.read_file(data_path + 'network/QGIS_Project/referentiel-comptages-edit.shp')
paris_districts = gpd.read_file(data_path + 'districts_paris.geojson')
df_car_detectors = gpd.read_file(data_path + 'all_car_detectors.geojson')

paris_districts = gpd.read_file('../../data/districts_paris.geojson')
polygon_all_districts = paris_districts.unary_union

def get_exterior_coords(df, start_point, end_point):
    filtered_gdf = df[(df["c_ar"] >= start_point) & (df["c_ar"] <= end_point)]

    # Check if there are any polygons matching the condition
    if not filtered_gdf.empty:
        # Apply unary_union to combine the selected polygons into a single polygon
        districts_polygon = unary_union(filtered_gdf["geometry"])
    else:
        # If no polygons match the condition, union_polygon will be None
        districts_polygon = None

    return districts_polygon.exterior.coords.xy

def get_polygon_geometry(df, start_point, end_point):
    filtered_gdf = df[(df["c_ar"] >= start_point) & (df["c_ar"] <= end_point)]

    # Check if there are any polygons matching the condition
    if not filtered_gdf.empty:
        # Apply unary_union to combine the selected polygons into a single polygon
        districts_polygon = unary_union(filtered_gdf["geometry"])
    else:
        # If no polygons match the condition, union_polygon will be None
        districts_polygon = None

    return districts_polygon

def transform_highway(value):
    if isinstance(value, list):
        return value[0] if value else None
    else:
        return value
    
def filter_for_district(x_coords, y_coords, df):
    district_polygon = Polygon(zip(x_coords, y_coords))

    # Create a GeoDataFrame containing the district polygon
    district_gdf = gpd.GeoDataFrame(geometry=[district_polygon], crs=df.crs)

    # Use the GeoDataFrame's cx attribute to spatially filter cycleways_2010_2022
    return gpd.overlay(df, district_gdf, how='intersection')


def is_na_list(lst):
    return lst is None or len(lst) == 0 or all(pd.isna(x) for x in lst)

def parse_and_average_lanes(lanes_str):
    if isinstance(lanes_str, list):
        if is_na_list(lanes_str):
            return np.nan
        else: 
            return sum(map(int, lanes_str)) / len(lanes_str)
    else:
        if pd.isna(lanes_str):  # Check if input is NaN
            return np.nan  # Return NaN if input is NaN
    try:
        # Attempt to parse the string as a list
        lanes_list = ast.literal_eval(lanes_str)
        if isinstance(lanes_list, list):
            # If it's a list, calculate the average of list elements
            return sum(map(int, lanes_list)) / len(lanes_list)
        else:
            # If it's a single integer, return it as is
            return int(lanes_list)
    except (SyntaxError, ValueError):
        # If parsing fails or the lanes_str is not a list, parse as single integer
        return int(lanes_str)

def line_length_in_meters(line_string):
    # Define a UTM projection for the zone containing your coordinates
    utm_zone = 31  # Assuming you are in Paris, which falls in UTM zone 31 for example
    proj = Proj(proj='utm', zone=utm_zone, ellps='WGS84')

    # Extract coordinates from the LineString
    coordinates = list(line_string.coords)

    # Transform the coordinates to UTM projection
    utm_coordinates = [proj(lon, lat) for lon, lat in coordinates]

    # Compute the distance between consecutive points in meters
    total_length = 0
    geod = Geod(ellps='WGS84')
    for i in range(len(utm_coordinates) - 1):
        lon1, lat1 = utm_coordinates[i]
        lon2, lat2 = utm_coordinates[i + 1]
        distance_meters = geod.inv(lon1, lat1, lon2, lat2)[-1]  # Use [-1] to get distance

        # Handle case of very small distances
        if np.isnan(distance_meters):
            dx = lon2 - lon1
            dy = lat2 - lat1
            distance_meters = np.sqrt(dx**2 + dy**2)
        total_length += distance_meters

    return total_length

def map_highway(df):
    highway_mapped = []
    for value in df['highway']:
        if isinstance(value, str):
            highway_mapped.append(value)
        elif isinstance(value, list):
            highway_mapped.append(value[0] if len(value) > 0 else None)
        else:
            highway_mapped.append(None)
    my_df = df.copy()
    my_df['highway_mapped'] = highway_mapped
    return my_df

## Abstract

The goal of this notebook is to investigate the OSM (historical) network of Paris, for different districts.  

In [2]:
x_district_1_4, y_district_1_4  = get_exterior_coords(paris_districts, 1, 4)
x_district_5_7, y_district_5_7  = get_exterior_coords(paris_districts, 5, 7)

In [5]:
alpha_shape = alphashape.alphashape(polygon_road_network, 435)
coordinates = list(alpha_shape.exterior[0].coords)
polygon = Polygon(coordinates)

G_2024 = ox.graph_from_polygon(polygon=polygon, simplify=True, network_type="drive")
nodes, edges_2024 = momepy.nx_to_gdf(G_2024, points=True, lines=True)

edges_2024['lanes_mapped'] = edges_2024['lanes'].apply(parse_and_average_lanes)
edges_2024 = map_highway(edges_2024)

average_lanes_per_highway_2024 = edges_2024.groupby('highway_mapped')['lanes_mapped'].mean()

/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_4087/1480800201.py:6: UserWarning: Approach is not set. Defaulting to 'primal'.
  nodes, edges_2024 = momepy.nx_to_gdf(G_2024, points=True, lines=True)


In [6]:
# set manually values for seldom highway classifications, which don't occur in the districts.

average_lanes_per_highway_2024['road'] = 2
average_lanes_per_highway_2024['virtual'] = 1

In [7]:
average_lanes_per_highway_2024

highway_mapped
living_street     1.284483
primary           3.077149
primary_link      1.844156
residential       1.441355
secondary         2.475537
secondary_link    1.636364
tertiary          2.072997
tertiary_link     1.285714
trunk             3.666667
trunk_link        1.780000
unclassified      1.677570
road              2.000000
virtual           1.000000
Name: lanes_mapped, dtype: float64

In [8]:
years = [2023, 2024]
zones = [1, 2]

def get_length_in_lane_km(df):
    length_in_lane_km = 0
    for idx, edge in df.iterrows():
        length = edge['length_computed']
        lanes = edge['lanes_mapped']
        length_edge = length * lanes/1000
        length_in_lane_km += length_edge
    return length_in_lane_km

for year in years:
    for zone in zones:
        if zone == 1:
            district = get_polygon_geometry(paris_districts, 1, 4)
        else: 
            district = get_polygon_geometry(paris_districts, 5, 7)
        overpass_settings = '[out:json][timeout:90][date:"' + str(year) + '-01-01T00:00:00Z"]'
        ox.settings.overpass_settings = overpass_settings
        ox.settings.log_console = True
    
        G = ox.graph_from_polygon(polygon=district, simplify=True, network_type="drive")
        nodes, edges = momepy.nx_to_gdf(G, points=True, lines=True)
        
        if 'lanes' not in edges.columns:
            edges['lanes'] = float('nan')
        edges['lanes_mapped'] = edges['lanes'].apply(parse_and_average_lanes)
        edges = map_highway(edges)  
   
        average_lanes_per_highway = edges.groupby('highway_mapped')['lanes_mapped'].mean()
        edges = edges[edges['geometry'].notnull()]
        edges['length_computed'] = edges['geometry'].apply(lambda x: line_length_in_meters(x))

        for index, row in edges.iterrows():
            if pd.isna(row['lanes_mapped']):
                approximated_lanes = average_lanes_per_highway[row['highway_mapped']]
                if (pd.isna(approximated_lanes)):
                    approximated_lanes = average_lanes_per_highway_2024[row['highway_mapped']]
                edges.at[index, 'lanes_mapped'] = approximated_lanes
        
        # filter for higher order roads
        edges_hor = edges[
            edges["highway"].str.contains("motorway") |
            edges["highway"].str.contains("trunk") |
            edges["highway"].str.contains("primary") |
            edges["highway"].str.contains("secondary") |
            edges["highway"].str.contains("tertiary") 
        ]
    
        length_in_km = edges['length_computed'].sum()/1000    
        length_in_lane_km = get_length_in_lane_km(edges)
        
        length_hor_in_km = edges_hor['length_computed'].sum()/1000
        length_in_lane_km_hor = get_length_in_lane_km(edges_hor)

        print(" ")
        print("Year: ", year, ", zone: ", zone)
        print("Length in km: " + str(length_in_km.round(2)))
        print("Length in lane km: " + str(length_in_lane_km))
        
        print("Length of higher order roads in km: " + str(length_hor_in_km.round(2)))
        print("Length of higher order roads in lane km: " + str(length_in_lane_km_hor))

/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_4087/1712476554.py:24: UserWarning: Approach is not set. Defaulting to 'primal'.
  nodes, edges = momepy.nx_to_gdf(G, points=True, lines=True)


 
Year:  2023 , zone:  1
Length in km: 84.25
Length in lane km: 169.99334976941216
Length of higher order roads in km: 28.75
Length of higher order roads in lane km: 83.15877582088862


/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_4087/1712476554.py:24: UserWarning: Approach is not set. Defaulting to 'primal'.
  nodes, edges = momepy.nx_to_gdf(G, points=True, lines=True)


 
Year:  2023 , zone:  2
Length in km: 157.93
Length in lane km: 311.76951463361
Length of higher order roads in km: 72.52
Length of higher order roads in lane km: 183.69892602185303


/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_4087/1712476554.py:24: UserWarning: Approach is not set. Defaulting to 'primal'.
  nodes, edges = momepy.nx_to_gdf(G, points=True, lines=True)


 
Year:  2024 , zone:  1
Length in km: 83.69
Length in lane km: 169.1203634431486
Length of higher order roads in km: 34.2
Length of higher order roads in lane km: 94.62613763209538


/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_4087/1712476554.py:24: UserWarning: Approach is not set. Defaulting to 'primal'.
  nodes, edges = momepy.nx_to_gdf(G, points=True, lines=True)


 
Year:  2024 , zone:  2
Length in km: 156.92
Length in lane km: 308.76602728909825
Length of higher order roads in km: 72.33
Length of higher order roads in lane km: 183.06721264183003
